In [ ]:
# %load ../../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath
import zcode.plot as zplot

def draw_gwb(ax, xx, gwb, nsamp=10, color=None, label=None):
    if color is None:
        color = ax._get_lines.get_next_color()

    mm, *conf = np.percentile(gwb, [50, 25, 75], axis=1)
    ax.plot(xx, mm, alpha=0.5, color=color, label=label)
    ax.fill_between(xx, *conf, color=color, alpha=0.15)

    if (nsamp is not None) and (nsamp > 0):
        nsamp_max = gwb.shape[1]
        idx = np.random.choice(nsamp_max, np.min([nsamp, nsamp_max]), replace=False)
        for ii in idx:
            ax.plot(xx, gwb[:, ii], color=color, alpha=0.25, lw=1.0, ls='-')
            
    return


In [ ]:
SHAPE = 20
TIME = 1e5 * GYR

gsmf = holo.sam.GSMF_Schechter()               # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()                 # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()                 # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()    # M-MBulge Relation            (MMB)

# hard = holo.hardening.Hard_GW()

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=SHAPE)
hard = holo.hardening.Fixed_Time.from_sam(sam, TIME,  exact=True)

In [ ]:
fobs_edges = utils.nyquist_freqs_edges(20*YR, 0.2*YR)
fobs = utils.midpoints(fobs_edges, log=True)
gwb = sam.gwb(fobs_edges, hard=hard, realize=30, )    # calculate many different realizations

In [ ]:
xx = fobs * YR
fig, ax = plot.figax()
draw_gwb(ax, xx, gwb)
# ax.plot(xx, gwb)

plot._twin_hz(ax)
plt.show()

# Calculate age during evolution from hardening model

In [ ]:
sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=None)
# hard = holo.hardening.Fixed_Time.from_sam(sam, GYR, exact=True)
hard = holo.hardening.Fixed_Time.from_sam(sam, GYR, exact=False)

STEPS = 100
# ()
rmax = hard._sepa_init
# (M,)
rmin = utils.rad_isco(sam.mtot)

extr = np.log10([rmax * np.ones_like(rmin), rmin])
rads = np.linspace(0.0, 1.0, STEPS)[np.newaxis, :]
# (M, X)
rads = extr[0][:, np.newaxis] + (extr[1] - extr[0])[:, np.newaxis] * rads
rads = 10.0 ** rads
mt, mr, rz, rads = np.broadcast_arrays(sam.mtot[:, np.newaxis, np.newaxis, np.newaxis], sam.mrat[np.newaxis, :, np.newaxis, np.newaxis], sam.redz[np.newaxis, np.newaxis, :, np.newaxis], rads[:, np.newaxis, np.newaxis, :])

mt, mr, rads = [mm.reshape(-1, STEPS).T for mm in [mt, mr, rads]]
for mm in [mt, mr, rads]:
    print(mm.shape)

dadt = hard.dadt(mt, mr, rads)
# Integrate (inverse) hardening rates to calculate total lifetime
times = -utils.trapz_loglog(-1.0 / dadt, rads, axis=0, cumsum=True)
print(rads.shape, times.shape)

In [ ]:
fig, ax = plot.figax()
tw = ax.twinx()
tw.set(yscale='log')

for ii in np.random.choice(rads.shape[1], 10, replace=False):
    ax.plot(rads[1:, ii]/PC, times[:, ii]/YR)
    tau = rads[:, ii] / -dadt[:, ii]
    tw.plot(rads[:, ii]/PC, tau/YR, ls='--')
    print(times[-1, ii]/GYR)
    print(rads[-1, ii]/PC)

plt.show()